In [138]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import numpy as np

## Web-Scraping der Kupfer-Daten

Die Daten der 'London Metal Exchange' sind leider nicht direkt abrufbar bzw. existiert auch keine API. Daher werden die Daten über eine Drittseite ((https://www.westmetall.com/en/westmetall.html), die täglich - mit Ausnahme von Sams- und Sonntagen sowie Feiertagen, an denen die Börse geschlossen hat - die neuesten Kursdaten der Londoner Metallbörse erfasst, geladen. Dazu wird ein Webscraper mittels BeautifulSoup aufgebaut und die Daten anschließend in das handliche Format eines pandas Dataframe gebracht.

Im Folgenden handelt es sich dabei um die Kursdaten für Kupfer.

In [130]:
# Scraping function to extract data from website

def scraping(c):
    r = requests.get(c)
    soup = BeautifulSoup(r.text,"html.parser")
    rows = soup.find_all('tr')
    list_rows = []
    
    for tr in rows:
        data = []
        for td in tr.find_all('td'):
            data.append(td.text)
        if data == []:
            continue
        list_rows.append(data)
    return list_rows

In [131]:
# Create dataframe from function

hyper = 'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash'

df = pd.DataFrame(scraping(hyper),columns=['date','lme_copper_cash','lme_copper_three_month','lme_copper_stock'])

In [111]:
df.head()

,date,lme_copper_cash,lme_copper_three_month,lme_copper_stock
0,2022-04-29,9820.50,9822.00,156050
1,2022-04-28,9812.00,9805.00,150850
2,2022-04-27,9910.00,9895.00,148500
3,2022-04-26,9912.50,9918.50,135175
4,2022-04-25,9919.00,9915.00,137275


In [132]:
# Change and convert datatypes: date column

df['date'] = pd.to_datetime(df['date'],format='%d. %B %Y')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3625 entries, 0 to 3624
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    3625 non-null   datetime64[ns]
 1   lme_copper_cash         3625 non-null   object        
 2   lme_copper_three_month  3625 non-null   object        
 3   lme_copper_stock        3625 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 113.4+ KB


In [143]:
# Change and convert datatypes: lme columns

df["lme_copper_cash"] = df["lme_copper_cash"].replace('-', np.NaN, regex=True)
df["lme_copper_stock"] = df["lme_copper_stock"].replace('-', np.NaN, regex=True)
df["lme_copper_three_month"] = df["lme_copper_three_month"].replace('-', np.NaN, regex=True)

df["lme_copper_cash"] = [float(str(i).replace(",", "")) for i in df["lme_copper_cash"]]
df["lme_copper_stock"] = [float(str(i).replace(",", "")) for i in df["lme_copper_stock"]]
df["lme_copper_three_month"] = [float(str(i).replace(",", "")) for i in df["lme_copper_three_month"]]

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3625 entries, 0 to 3624
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    3625 non-null   datetime64[ns]
 1   lme_copper_cash         3622 non-null   float64       
 2   lme_copper_three_month  3622 non-null   float64       
 3   lme_copper_stock        3624 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 141.6 KB


In [145]:
df.head(20)

,date,lme_copper_cash,lme_copper_three_month,lme_copper_stock
0,2022-04-29,9820.5,9822.0,156050.0
1,2022-04-28,9812.0,9805.0,150850.0
2,2022-04-27,9910.0,9895.0,148500.0
3,2022-04-26,9912.5,9918.5,135175.0
4,2022-04-25,9919.0,9915.0,137275.0
5,2022-04-22,10230.5,10235.0,137775.0
6,2022-04-21,10268.0,10282.0,130500.0
7,2022-04-20,10215.0,10250.0,128775.0
8,2022-04-19,10381.0,10391.5,118825.0
9,2022-04-14,10277.0,10310.0,110675.0


In [146]:
#Export to csv

df.to_csv('dataset_copper.csv')

## Einlesen der BIP-Daten

### China

In [2]:
df_bip_china = pd.read_excel("C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\BIP\\China Bruttoinlandsprodukt pro Kopf in jeweiligen Preisen von 1980 bis 2020 und Prognosen bis 2026.xlsx",sheet_name="Daten", skiprows=4)

In [3]:
df_bip_china = df_bip_china.drop('Unnamed: 0', axis = 1)
df_bip_china.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CN_BIP pro Kopf in USD'}, inplace=True)

In [4]:
df_bip_china['Jahr'] = df_bip_china['Jahr'].astype(int)
df_bip_china.head()

,Jahr,CN_BIP pro Kopf in USD
0,1980,306.98
1,1981,288.49
2,1982,279.97
3,1983,296.51
4,1984,301.11


In [5]:
df_bip_china.dtypes

Jahr                        int32
CN_BIP pro Kopf in USD    float64
dtype: object

### USA

In [6]:
df_bip_usa = pd.read_excel("C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\BIP\\USA Bruttoinlandsprodukt pro Kopf in jeweiligen Preisen von 1980 bis 2020 und Prognosen bis 2026.xlsx",sheet_name="Daten", skiprows=4)

In [7]:
df_bip_usa = df_bip_usa.drop('Unnamed: 0', axis = 1)
df_bip_usa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'US_BIP pro Kopf in USD'}, inplace=True)

In [8]:
df_bip_usa.head()

,Jahr,US_BIP pro Kopf in USD
0,1980,12552.94
1,1981,13948.70
2,1982,14404.99
3,1983,15513.68
4,1984,17086.44


In [9]:
df_bip_usa.dtypes

Jahr                        int64
US_BIP pro Kopf in USD    float64
dtype: object

### Einlesen der Kupfernachfrage

In [10]:
df_nachfrage_weltweit = pd.read_excel("C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Nachfrage\\Weltweite_Kupfernachfrage_2011-2021.xlsx", sheet_name='Daten', skiprows=4)

In [11]:
df_nachfrage_weltweit= df_nachfrage_weltweit.drop('Unnamed: 0', axis = 1)
df_nachfrage_weltweit.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'Global_Nachfrage in Mio. Tonnen'}, inplace=True)

df_nachfrage_weltweit.head()

,Jahr,Global_Nachfrage in Mio. Tonnen
0,2011,19.71
1,2012,20.48
2,2013,21.42
3,2014,22.93
4,2015,23.06


In [12]:
df_nachfrage_weltweit.dtypes

Jahr                                 int64
Global_Nachfrage in Mio. Tonnen    float64
dtype: object

## Einlesen der Produktionsmengen

In [13]:
Produktionsmengen_Europa = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Kupferproduktion_Europa_1995-2020.xlsx"
Produktionsmengen_Australien = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Australien_2009-2021.xlsx"
Produktionsmengen_Chile = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Chile_2009-2021.xlsx"
Produktionsmengen_China = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_China_2009-2021.xlsx"
Produktionsmengen_Kasachstan = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Kasachstan_2009-2021.xlsx"
Produktionsmengen_Mexiko = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Mexiko_2009-2021.xlsx"
Produktionsmengen_Peru = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Peru_2009-2021.xlsx"
Produktionsmengen_Russland = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Russland_2009-2021.xlsx"
Produktionsmengen_Sambia = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_Sambia_2009-2021.xlsx"
Produktionsmenge_USA = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Produktion\\Minenproduktion_Kupfer_USA_2009-2021.xlsx"

### Einlesen der Dateien

In [14]:
df_produktion_europa = pd.read_excel(Produktionsmengen_Europa, sheet_name='Daten', skiprows=4)
df_produktion_australien = pd.read_excel(Produktionsmengen_Australien, sheet_name='Daten', skiprows=4)
df_produktion_chile = pd.read_excel(Produktionsmengen_Chile, sheet_name='Daten', skiprows=4)
df_produktion_china = pd.read_excel(Produktionsmengen_China, sheet_name='Daten', skiprows=4)
df_produktion_kasachstan = pd.read_excel(Produktionsmengen_Kasachstan, sheet_name='Daten', skiprows=4)
df_produktion_mexiko = pd.read_excel(Produktionsmengen_Mexiko, sheet_name='Daten', skiprows=4)
df_produktion_peru = pd.read_excel(Produktionsmengen_Peru, sheet_name='Daten', skiprows=4)
df_produktion_russland = pd.read_excel(Produktionsmengen_Russland, sheet_name='Daten', skiprows=4)
df_produktion_sambia = pd.read_excel(Produktionsmengen_Sambia, sheet_name='Daten', skiprows=4)
df_produktion_usa = pd.read_excel(Produktionsmenge_USA, sheet_name='Daten', skiprows=4)

### Vorbereitung der Daten

In [15]:
df_produktion_europa.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,1995,3261
1,NaN,1996,3240
2,NaN,1997,3389
3,NaN,1998,3571
4,NaN,1999,3493


In [16]:
# Europa
df_produktion_europa= df_produktion_europa.drop('Unnamed: 0', axis = 1)
df_produktion_europa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'EU_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# Australien
df_produktion_australien= df_produktion_australien.drop('Unnamed: 0', axis = 1)
df_produktion_australien.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'AU_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# Chile
df_produktion_chile= df_produktion_chile.drop('Unnamed: 0', axis = 1)
df_produktion_chile.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CL_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# China
df_produktion_china= df_produktion_china.drop('Unnamed: 0', axis = 1)
df_produktion_china.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CN_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# Kasachstan
df_produktion_kasachstan= df_produktion_kasachstan.drop('Unnamed: 0', axis = 1)
df_produktion_kasachstan.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'KZ_Produktionsmenge in 1000 Tonnen'}, inplace=True)


# Mexiko
df_produktion_mexiko= df_produktion_mexiko.drop('Unnamed: 0', axis = 1)
df_produktion_mexiko.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'MX_Produktionsmenge in 1000 Tonnen'}, inplace=True)

#Peru
df_produktion_peru= df_produktion_peru.drop('Unnamed: 0', axis = 1)
df_produktion_peru.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'PE_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# Russland
df_produktion_russland= df_produktion_russland.drop('Unnamed: 0', axis = 1)
df_produktion_russland.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'RU_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# Sambia
df_produktion_sambia= df_produktion_sambia.drop('Unnamed: 0', axis = 1)
df_produktion_sambia.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'ZM_Produktionsmenge in 1000 Tonnen'}, inplace=True)

# USA
df_produktion_usa= df_produktion_usa.drop('Unnamed: 0', axis = 1)
df_produktion_usa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'US_Produktionsmenge in 1000 Tonnen'}, inplace=True)



In [17]:
df_produktion_usa.dtypes

Jahr                                  int64
US_Produktionsmenge in 1000 Tonnen    int64
dtype: object

## Einlesen der Verbrauchsdaten

### China

In [18]:
Pfad_cn = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Verbrauch\\Kupferverbrauch_China_2001-2020.xlsx"

In [19]:
df_verbrauch_china = pd.read_excel(Pfad_cn, sheet_name='Daten', skiprows=4)

In [20]:
df_verbrauch_china= df_verbrauch_china.drop('Unnamed: 0', axis = 1)
df_verbrauch_china.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CN_Verbrauch in 1000 Tonnen'}, inplace=True)

df_verbrauch_china.head()

,Jahr,CN_Verbrauch in 1000 Tonnen
0,2001,2251
1,2002,2601
2,2003,3115
3,2004,3487
4,2005,3775


### Europa

In [21]:
Pfad_eu = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Verbrauch\\Kupferverbrauch_Europa_1995-2020.xlsx"

In [22]:
df_verbrauch_europa = pd.read_excel(Pfad_eu, sheet_name='Daten', skiprows=4)

In [23]:
df_verbrauch_europa= df_verbrauch_europa.drop('Unnamed: 0', axis = 1)
df_verbrauch_europa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'EU_Verbrauch in 1000 Tonnen'}, inplace=True)

df_verbrauch_europa.head()

,Jahr,EU_Verbrauch in 1000 Tonnen
0,1995,3091
1,1996,3070
2,1997,3177
3,1998,3351
4,1999,3226


### Japan

In [24]:
Pfad_jp = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Verbrauch\\Kupferverbrauch_Japan_2001-2020.xlsx"

In [25]:
df_verbrauch_japan = pd.read_excel(Pfad_jp, sheet_name='Daten', skiprows=4)

In [26]:
df_verbrauch_japan= df_verbrauch_japan.drop('Unnamed: 0', axis = 1)
df_verbrauch_japan.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'JP_Verbrauch in 1000 Tonnen'}, inplace=True)

df_verbrauch_japan.head()

,Jahr,JP_Verbrauch in 1000 Tonnen
0,2001,1100
1,2002,1042
2,2003,1049
3,2004,1116
4,2005,1132


In [27]:
df_verbrauch_japan.dtypes

Jahr                           int64
JP_Verbrauch in 1000 Tonnen    int64
dtype: object

### USA

In [28]:
Pfad_us = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Verbrauch\\Kupferverbrauch_USA_2001-2020.xlsx"

In [29]:
df_verbrauch_usa = pd.read_excel(Pfad_us, sheet_name='Daten', skiprows=4)

In [30]:
df_verbrauch_usa= df_verbrauch_usa.drop('Unnamed: 0', axis = 1)
df_verbrauch_usa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'US_Verbrauch in 1000 Tonnen'}, inplace=True)
df_verbrauch_usa['Jahr'] = df_verbrauch_usa['Jahr'].astype(int)
df_verbrauch_usa

,Jahr,US_Verbrauch in 1000 Tonnen
0,2001,2724
1,2002,2459
2,2003,2548
3,2004,2728
4,2005,2558
5,2006,2631
6,2007,2473
7,2008,2183
8,2009,1660
9,2010,1750


In [31]:
df_verbrauch_usa.dtypes

Jahr                           int32
US_Verbrauch in 1000 Tonnen    int64
dtype: object

## Einlesen der Reservedaten (Lagerbestände/Vorräte)

In [32]:
Res_Australien = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Australien in den Jahren 2010 bis 2021.xlsx"
Res_Chile = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Chile in den Jahren 2010 bis 2021.xlsx"
Res_China = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in China in den Jahren 2010 bis 2021.xlsx"
Res_USA = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in den USA in den Jahren 2010 bis 2021.xlsx"
Res_Kongo = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in der Demokratischen Republik Kongo in den Jahren 2012 bis 2021.xlsx"
Res_Kasachstan = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Kasachstan in den Jahren 2010 bis 2021.xlsx"
Res_Mexiko = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Mexiko in den Jahren 2010 bis 2021.xlsx"
Res_Peru = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Peru in den Jahren 2010 bis 2021.xlsx"
Res_Russland = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Russland in den Jahren 2010 bis 2021.xlsx"
Res_Sambia = "C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\Reserven\\Kupferreserven in Sambia in den Jahren 2010 bis 2021.xlsx"

### Ablage der Dataframes und Einlesen der Excel-Files

In [33]:
df_res_australien = pd.read_excel(Res_Australien, sheet_name='Daten', skiprows=4)
df_res_chile = pd.read_excel(Res_Chile, sheet_name='Daten', skiprows=4)
df_res_china = pd.read_excel(Res_China, sheet_name='Daten', skiprows=4)
df_res_usa = pd.read_excel(Res_USA, sheet_name='Daten', skiprows=4)
df_res_kongo = pd.read_excel(Res_Kongo, sheet_name='Daten', skiprows=4)
df_res_kasachstan = pd.read_excel(Res_Kasachstan, sheet_name='Daten', skiprows=4)
df_res_mexiko = pd.read_excel(Res_Mexiko, sheet_name='Daten', skiprows=4)
df_res_peru = pd.read_excel(Res_Peru, sheet_name='Daten', skiprows=4)
df_res_russland = pd.read_excel(Res_Russland, sheet_name='Daten', skiprows=4)
df_res_sambia = pd.read_excel(Res_Sambia, sheet_name='Daten', skiprows=4)

In [34]:
df_res_chile.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,2010,150
1,NaN,2011,190
2,NaN,2012,190
3,NaN,2013,190
4,NaN,2014,209


In [35]:
# Australien
df_res_australien= df_res_australien.drop('Unnamed: 0', axis = 1)
df_res_australien.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'AU_Reserven in Mio. Tonnen'}, inplace=True)
# Chile
df_res_chile= df_res_chile.drop('Unnamed: 0', axis = 1)
df_res_chile.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CL_Reserven in Mio. Tonnen'}, inplace=True)
# China
df_res_china= df_res_china.drop('Unnamed: 0', axis = 1)
df_res_china.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CN_Reserven in Mio. Tonnen'}, inplace=True)

# USA
df_res_usa= df_res_usa.drop('Unnamed: 0', axis = 1)
df_res_usa.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'US_Reserven in Mio. Tonnen'}, inplace=True)

# Kongo
df_res_kongo= df_res_kongo.drop('Unnamed: 0', axis = 1)
df_res_kongo.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'CG_Reserven in Mio. Tonnen'}, inplace=True)

# Kasachstan
df_res_kasachstan= df_res_kasachstan.drop('Unnamed: 0', axis = 1)
df_res_kasachstan.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'KZ_Reserven in Mio. Tonnen'}, inplace=True)

# Mexiko
df_res_mexiko= df_res_mexiko.drop('Unnamed: 0', axis = 1)
df_res_mexiko.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'MX_Reserven in Mio. Tonnen'}, inplace=True)

# Peru
df_res_peru= df_res_peru.drop('Unnamed: 0', axis = 1)
df_res_peru.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'PE_Reserven in Mio. Tonnen'}, inplace=True)

# Russland
df_res_russland= df_res_russland.drop('Unnamed: 0', axis = 1)
df_res_russland.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'RU_Reserven in Mio. Tonnen'}, inplace=True)

# Sambia
df_res_sambia= df_res_sambia.drop('Unnamed: 0', axis = 1)
df_res_sambia.rename(columns={'Unnamed: 1': 'Jahr',
                           'Unnamed: 2': 'ZM_Reserven in Mio. Tonnen'}, inplace=True)

## Joinen der Datensätze

Folgende Datensätze müssen miteinander, auf der Spalte "Jahr" gejoint werden:

In [36]:
# Liste aller DataFrames
df_list = [df_bip_china,
df_bip_usa,
df_nachfrage_weltweit,
df_produktion_europa,
df_produktion_australien,
df_produktion_chile,
df_produktion_china,
df_produktion_kasachstan,
df_produktion_mexiko,
df_produktion_peru,
df_produktion_russland,
df_produktion_sambia,
df_produktion_usa,
df_verbrauch_china,
df_verbrauch_europa,
df_verbrauch_japan,
df_verbrauch_usa,
df_res_australien,
df_res_chile,
df_res_china,
df_res_usa,
df_res_kongo,
df_res_kasachstan,
df_res_mexiko,
df_res_peru,
df_res_russland,
df_res_sambia]


In [37]:
# Überprüfung der Datentypen, insbesondere der Schlüsselspalte "Jahr"
for i in df_list:
    print(i.dtypes)

Jahr                        int32
CN_BIP pro Kopf in USD    float64
dtype: object
Jahr                        int64
US_BIP pro Kopf in USD    float64
dtype: object
Jahr                                 int64
Global_Nachfrage in Mio. Tonnen    float64
dtype: object
Jahr                                  int64
EU_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
AU_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
CL_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
CN_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
KZ_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
MX_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                                  int64
PE_Produktionsmenge in 1000 Tonnen    int64
dtype: object
Jahr                  

In [38]:
from functools import reduce

In [39]:
# Joinen der DataFrames aus o.g. Liste auf der Schlüsselspalte "Jahr"
df_final = reduce(lambda left,right: pd.merge(left,right, how = 'outer', on='Jahr'), df_list)

In [40]:
df_final

,Jahr,CN_BIP pro Kopf in USD,US_BIP pro Kopf in USD,Global_Nachfrage in Mio. Tonnen,EU_Produktionsmenge in 1000 Tonnen,AU_Produktionsmenge in 1000 Tonnen,CL_Produktionsmenge in 1000 Tonnen,CN_Produktionsmenge in 1000 Tonnen,KZ_Produktionsmenge in 1000 Tonnen,MX_Produktionsmenge in 1000 Tonnen,...,AU_Reserven in Mio. Tonnen,CL_Reserven in Mio. Tonnen,CN_Reserven in Mio. Tonnen,US_Reserven in Mio. Tonnen,CG_Reserven in Mio. Tonnen,KZ_Reserven in Mio. Tonnen,MX_Reserven in Mio. Tonnen,PE_Reserven in Mio. Tonnen,RU_Reserven in Mio. Tonnen,ZM_Reserven in Mio. Tonnen
0,1980,306.98,12552.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1981,288.49,13948.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1982,279.97,14404.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1983,296.51,15513.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1984,301.11,17086.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1985,292.99,18199.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1986,279.91,19034.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1987,299.84,20000.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1988,368.08,21376.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1989,406.53,22814.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df_final2 = df_final[df_final['Jahr']>=2012]
df_final3 = df_final2[df_final2['Jahr']<=2020]

In [42]:
df_final3

,Jahr,CN_BIP pro Kopf in USD,US_BIP pro Kopf in USD,Global_Nachfrage in Mio. Tonnen,EU_Produktionsmenge in 1000 Tonnen,AU_Produktionsmenge in 1000 Tonnen,CL_Produktionsmenge in 1000 Tonnen,CN_Produktionsmenge in 1000 Tonnen,KZ_Produktionsmenge in 1000 Tonnen,MX_Produktionsmenge in 1000 Tonnen,...,AU_Reserven in Mio. Tonnen,CL_Reserven in Mio. Tonnen,CN_Reserven in Mio. Tonnen,US_Reserven in Mio. Tonnen,CG_Reserven in Mio. Tonnen,KZ_Reserven in Mio. Tonnen,MX_Reserven in Mio. Tonnen,PE_Reserven in Mio. Tonnen,RU_Reserven in Mio. Tonnen,ZM_Reserven in Mio. Tonnen
32,2012,6282.71,51736.74,20.48,3882.0,958.0,5430.0,1630.0,424.0,440.0,...,86.0,190.0,30.0,39.0,20.0,7.0,38.0,76.0,30.0,20.0
33,2013,7039.57,53245.52,21.42,3950.0,990.0,5780.0,1600.0,446.0,480.0,...,87.0,190.0,30.0,39.0,20.0,7.0,38.0,70.0,30.0,20.0
34,2014,7645.88,55083.51,22.93,3960.0,970.0,5750.0,1760.0,430.0,515.0,...,93.0,209.0,30.0,35.0,20.0,6.0,38.0,68.0,30.0,20.0
35,2015,8034.29,56729.68,23.06,3704.0,971.0,5760.0,1710.0,NaN,594.0,...,88.0,210.0,30.0,33.0,20.0,NaN,46.0,82.0,30.0,20.0
36,2016,8063.45,57839.99,23.49,3634.0,948.0,5550.0,1900.0,NaN,752.0,...,89.0,210.0,28.0,35.0,20.0,NaN,46.0,81.0,30.0,20.0
37,2017,8760.26,59885.71,23.69,3682.0,860.0,5500.0,1710.0,NaN,742.0,...,88.0,170.0,27.0,45.0,20.0,NaN,46.0,81.0,NaN,20.0
38,2018,9848.95,62769.66,24.48,3769.0,920.0,5830.0,1590.0,603.0,751.0,...,88.0,170.0,26.0,48.0,20.0,NaN,50.0,83.0,61.0,19.0
39,2019,10170.06,65051.88,24.41,3641.0,934.0,5790.0,1680.0,562.0,715.0,...,87.0,200.0,26.0,51.0,19.0,20.0,53.0,87.0,61.0,19.0
40,2020,10511.34,63358.49,24.99,3332.0,885.0,5730.0,1720.0,552.0,733.0,...,88.0,200.0,26.0,48.0,19.0,20.0,53.0,92.0,61.0,21.0


In [43]:
df_final3.isnull().sum().sort_values()

Jahr                                  0
PE_Reserven in Mio. Tonnen            0
MX_Reserven in Mio. Tonnen            0
CG_Reserven in Mio. Tonnen            0
US_Reserven in Mio. Tonnen            0
CN_Reserven in Mio. Tonnen            0
CL_Reserven in Mio. Tonnen            0
AU_Reserven in Mio. Tonnen            0
US_Verbrauch in 1000 Tonnen           0
JP_Verbrauch in 1000 Tonnen           0
EU_Verbrauch in 1000 Tonnen           0
CN_Verbrauch in 1000 Tonnen           0
US_Produktionsmenge in 1000 Tonnen    0
RU_Produktionsmenge in 1000 Tonnen    0
PE_Produktionsmenge in 1000 Tonnen    0
MX_Produktionsmenge in 1000 Tonnen    0
CN_Produktionsmenge in 1000 Tonnen    0
CL_Produktionsmenge in 1000 Tonnen    0
AU_Produktionsmenge in 1000 Tonnen    0
EU_Produktionsmenge in 1000 Tonnen    0
Global_Nachfrage in Mio. Tonnen       0
US_BIP pro Kopf in USD                0
CN_BIP pro Kopf in USD                0
ZM_Produktionsmenge in 1000 Tonnen    0
ZM_Reserven in Mio. Tonnen            0


In [44]:
# Aufgrund der geringen Bedeutung von Kasachstan bei der Menge der Kupferreserven und Kupferproduktion und der Unvollständigkeit der Daten, wird Kasachstan aus der Analyse entfernt
df_final4 = df_final3.drop('KZ_Reserven in Mio. Tonnen',axis=1)
df_final5 = df_final4.drop('KZ_Produktionsmenge in 1000 Tonnen',axis=1)

In [45]:
df_final5

,Jahr,CN_BIP pro Kopf in USD,US_BIP pro Kopf in USD,Global_Nachfrage in Mio. Tonnen,EU_Produktionsmenge in 1000 Tonnen,AU_Produktionsmenge in 1000 Tonnen,CL_Produktionsmenge in 1000 Tonnen,CN_Produktionsmenge in 1000 Tonnen,MX_Produktionsmenge in 1000 Tonnen,PE_Produktionsmenge in 1000 Tonnen,...,US_Verbrauch in 1000 Tonnen,AU_Reserven in Mio. Tonnen,CL_Reserven in Mio. Tonnen,CN_Reserven in Mio. Tonnen,US_Reserven in Mio. Tonnen,CG_Reserven in Mio. Tonnen,MX_Reserven in Mio. Tonnen,PE_Reserven in Mio. Tonnen,RU_Reserven in Mio. Tonnen,ZM_Reserven in Mio. Tonnen
32,2012,6282.71,51736.74,20.48,3882.0,958.0,5430.0,1630.0,440.0,1300.0,...,1832.0,86.0,190.0,30.0,39.0,20.0,38.0,76.0,30.0,20.0
33,2013,7039.57,53245.52,21.42,3950.0,990.0,5780.0,1600.0,480.0,1380.0,...,1828.0,87.0,190.0,30.0,39.0,20.0,38.0,70.0,30.0,20.0
34,2014,7645.88,55083.51,22.93,3960.0,970.0,5750.0,1760.0,515.0,1380.0,...,1827.0,93.0,209.0,30.0,35.0,20.0,38.0,68.0,30.0,20.0
35,2015,8034.29,56729.68,23.06,3704.0,971.0,5760.0,1710.0,594.0,1700.0,...,1829.0,88.0,210.0,30.0,33.0,20.0,46.0,82.0,30.0,20.0
36,2016,8063.45,57839.99,23.49,3634.0,948.0,5550.0,1900.0,752.0,2350.0,...,1869.0,89.0,210.0,28.0,35.0,20.0,46.0,81.0,30.0,20.0
37,2017,8760.26,59885.71,23.69,3682.0,860.0,5500.0,1710.0,742.0,2450.0,...,1859.0,88.0,170.0,27.0,45.0,20.0,46.0,81.0,NaN,20.0
38,2018,9848.95,62769.66,24.48,3769.0,920.0,5830.0,1590.0,751.0,2440.0,...,1942.0,88.0,170.0,26.0,48.0,20.0,50.0,83.0,61.0,19.0
39,2019,10170.06,65051.88,24.41,3641.0,934.0,5790.0,1680.0,715.0,2460.0,...,1920.0,87.0,200.0,26.0,51.0,19.0,53.0,87.0,61.0,19.0
40,2020,10511.34,63358.49,24.99,3332.0,885.0,5730.0,1720.0,733.0,2150.0,...,1873.0,88.0,200.0,26.0,48.0,19.0,53.0,92.0,61.0,21.0


In [46]:
df_final5[['Jahr','RU_Reserven in Mio. Tonnen','Global_Nachfrage in Mio. Tonnen', 'CG_Reserven in Mio. Tonnen' ]]

,Jahr,RU_Reserven in Mio. Tonnen,Global_Nachfrage in Mio. Tonnen,CG_Reserven in Mio. Tonnen
32,2012,30.0,20.48,20.0
33,2013,30.0,21.42,20.0
34,2014,30.0,22.93,20.0
35,2015,30.0,23.06,20.0
36,2016,30.0,23.49,20.0
37,2017,NaN,23.69,20.0
38,2018,61.0,24.48,20.0
39,2019,61.0,24.41,19.0
40,2020,61.0,24.99,19.0


In [47]:
df_final5['RU_Reserven in Mio. Tonnen'] = df_final5['RU_Reserven in Mio. Tonnen'].fillna('30.0')

In [48]:
df_final5.isnull().sum().sort_values()

Jahr                                  0
PE_Reserven in Mio. Tonnen            0
MX_Reserven in Mio. Tonnen            0
CG_Reserven in Mio. Tonnen            0
US_Reserven in Mio. Tonnen            0
CN_Reserven in Mio. Tonnen            0
CL_Reserven in Mio. Tonnen            0
AU_Reserven in Mio. Tonnen            0
US_Verbrauch in 1000 Tonnen           0
JP_Verbrauch in 1000 Tonnen           0
EU_Verbrauch in 1000 Tonnen           0
CN_Verbrauch in 1000 Tonnen           0
US_Produktionsmenge in 1000 Tonnen    0
ZM_Produktionsmenge in 1000 Tonnen    0
RU_Produktionsmenge in 1000 Tonnen    0
PE_Produktionsmenge in 1000 Tonnen    0
MX_Produktionsmenge in 1000 Tonnen    0
CN_Produktionsmenge in 1000 Tonnen    0
CL_Produktionsmenge in 1000 Tonnen    0
AU_Produktionsmenge in 1000 Tonnen    0
EU_Produktionsmenge in 1000 Tonnen    0
Global_Nachfrage in Mio. Tonnen       0
US_BIP pro Kopf in USD                0
CN_BIP pro Kopf in USD                0
RU_Reserven in Mio. Tonnen            0


In [50]:
# Export des DataFrames als CSV-Datei
df_final5.to_csv("C:\\Users\\woelk\\OneDrive\\Dokumente\\Studium - HdM Stuttgart\\Machine Learning\\0 - Projekt\\Kupferpreisprognose\\copper_economic.csv", sep=",", encoding = 'utf-8')